# Download and Unzip Data Subset
If you are using your own shared drive with the tar file of images that you would like to use for training make sure to place the id number after the `gdown` command. For example if your shared url is:

https://drive.google.com/uc?id=1kvMIz2HryYLU_eek7w7lj_-xYSKd8rEL

your id is: 1kvMIz2HryYLU_eek7w7lj_-xYSKd8rEL

Notice that unix shell commands are preceded with an `!` in the jupyter notebook.

In [ ]:
# Uncomment for the 4hr-hiGy Fe&Xray subset
# !gdown 1kvMIz2HryYLU_eek7w7lj_-xYSKd8rEL

# Uncomment for the full dataset
# !gdown 1-h0SCkI2kjJlLV_PiHYYZZVkEjnDJ9qd

In [ ]:
# Unzip files using the shell command
!tar -xvf bps_hi_4hr.tar    # Our subset
# !tar -xvf full_dataset.tar  # Full dataset (~2GB)

# Alternative 1 - Use your own Drive
## Define a subset and split, then directly download the split from the AWS bucket for the [Biological and Physical Sciences (BPS) Microscopy Benchmark Training Dataset](https://registry.opendata.aws/bps_microscopy/)

In [ ]:
from google.colab import files    # downloading/uploading from/to your personal Drive
import pandas as pd               # Storing the data in a dataframe
import io
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from io import BytesIO
import os

In [ ]:
# Navigate the UI that pops up after running the cell to get your zip/tar
subset_from_drive = files.upload()
filename = next(iter(subset_from_drive))

if filename.endswith('.tar')
  !tar -xvf $filename
elif filename.endswith('.zip')
  !unzip $filename

# Alternative 2 - Create your own subset
- You can do this using the meta.csv provided for the full dataset

In [ ]:
!gdown 1hvk0zqmV_JGyxta99lRfxksUyhMz3rt1

In [ ]:
def export_subset_meta_dose_hr(
    dose_Gy_specifier: list,
    hr_post_exposure_val: list,
    in_csv_path_local: str,             # path includes name of file w/ extension
    out_dir_csv: str) -> tuple:
    """
    This function opens a csv file that contains the filenames of the bps microscopy data from the
    s3 bucket saved either locally or as a file_buffer object as a pandas dataframe. The dataframe
    is then sliced over the attributes of interest and written to another csv file for data
    versioning.

    args:
      dose_Gy (list): dose_Gy is a string corresponding to the dose of interest ['hi', 'med', 'low']
      hr_post_exposure_val (list): hr_post_exposure_val is an integer corresponding to the hour post
      exposure of interest [4, 24, 48]
      in_csv_path_local (str): a string of input original csv file
      out_dir_csv (str): a string of the output directory you would like to write the subset_meta file to

    returns:
      Tuple[str, int]: a tuple of the output csv file path and the number of rows in the output csv
      file
    """
    # Create output directory out_dir_csv if it does not exist
    if not os.path.exists(out_dir_csv):
        #if not, make one
        os.makedirs(out_dir_csv)


    # Load csv file into pandas DataFrame
    csv_data_frame = pd.read_csv(in_csv_path_local)

    # Check that dose_Gy and hr_post_exposure_val are valid

    #               low, med, hi
    # Fe dose_Gy = [0.0, 0.3, 0.82]
    # Xray dose_Gy = [0.0, 0.1, 1.0]
    if ((csv_data_frame["particle_type"] == "Fe") & (~csv_data_frame["dose_Gy"].isin([0.0, 0.3, 0.82]))).any():
        raise Exception("One or more Fe dose values are not valid")

    if ((csv_data_frame["particle_type"] == "X-ray") & (~csv_data_frame["dose_Gy"].isin([0.0, 0.1, 1.0]))).any():
        raise Exception("One or more X-ray dose values are not valid")

    if ((~csv_data_frame["hr_post_exposure"].isin([4, 24, 48]))).any():     # ~ is the bitwise NOT operator. Flips the bits of the operand.
        raise Exception("One or more exposure values are not valid")

    # Slice DataFrame by attributes of interest
    Fe_dose_gy = {'low': 0.0, 'med': 0.3, 'hi': 0.82}
    Xray_dose_gy = {'low': 0.0, 'med': 0.1, 'hi': 1.0}

    # Create a list of the dose_Gy values corresponding to the dose_Gy_specifier
    temp_list_Fe = []
    temp_list_Xray = []

    for specifier in dose_Gy_specifier:
        temp_list_Fe.append(Fe_dose_gy[specifier])
        temp_list_Xray.append(Xray_dose_gy[specifier])

    csv_data_frame = csv_data_frame[
        # Add every row where the hr_post_exposure value is in the argument list passed in
        (csv_data_frame["hr_post_exposure"].isin(hr_post_exposure_val)) &
        (
            ((csv_data_frame["particle_type"] == "Fe") & (csv_data_frame["dose_Gy"].isin(temp_list_Fe))) |
            ((csv_data_frame["particle_type"] == "X-ray") & (csv_data_frame["dose_Gy"].isin(temp_list_Xray)))
        )
    ]


    hr_post_val_string = '_'.join(str(item) for item in hr_post_exposure_val)
    gy_dose_string = '_'.join(str(item) for item in dose_Gy_specifier)

    # Write sliced DataFrame to output csv file with same name as input csv file with
    # _dose_hr_post_exposure.csv appended
    file = os.path.splitext(in_csv_path_local)[0]
    new_file_path = file + "_dose_" + gy_dose_string + "_hr_" + hr_post_val_string + "_post_exposure.csv"

    # Construct output csv file path using out_dir_csv and the name of the input csv file
    # with the dose_Gy and hr_post_exposure_val appended to the name of the input csv file
    # for data versioning.
    csv_data_frame.to_csv(new_file_path)

    # Write sliced DataFrame to output csv file with name constructed above
    return(new_file_path, csv_data_frame.shape[0])

In [ ]:
output_dir = 'subset_csv'

# Change arguments to match your desired subset for particle type classification
# By default it has the entire dataset
subset_new_path_fname, subset_size = export_subset_meta_dose_hr(
    dose_Gy_specifier=['low', 'med', 'hi'],
    hr_post_exposure_val=[4, 24, 48],
    in_csv_path_local='meta.csv',
    out_dir_csv=output_dir)

In [ ]:
subset_new_path_fname

In [ ]:
subset_size

## Now that you have a subset, we need to create a train/test split

In [ ]:
def train_test_split_subset_meta_dose_hr(
        subset_meta_dose_hr_csv_path: str,
        test_size: float,
        out_dir_csv: str,
        random_state: int = None,
        stratify_col: str = None
        ) -> tuple:
    """
    This function reads in a csv file containing the filenames of the bps microscopy data for
    a subset selected by the dose_Gy and hr_post_exposure attributes. The function then opens
    the file as a pandas dataframe and splits the dataframe into train and test sets using
    sklearn.model_selection.train_test_split. The train and test dataframes are then exported
    to csv files in the same directory as the input csv file.

    args:
        subset_meta_dose_hr_csv_path (str): a string of the input csv file path (full path includes filename)
        test_size (float or int): a float between 0 and 1 corresponding to the proportion of the data
        that should be in the test set. If int, represents the absolute number of test samples.
        out_dir_csv (str): a string of the output directory you would like to write the train and test
        random_state (int, RandomState instance or None, optional): controls the shuffling
        applied to the data before applying the split. Pass an int for reproducible output
        across multiple function calls.
        stratify (array-like or None, optional): array containing the labels for stratification.
        Default: None.
    returns:
        Tuple[str, str]: a tuple of the output csv file paths for the train and test sets
    """
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    # Load csv file into pandas DataFrame and use the train_test_split function to split the
    # DataFrame into train and test sets
    df = pd.read_csv(subset_meta_dose_hr_csv_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[stratify_col])

    # Rewrite index numbers for both train and test sets to conform to order in new dataframe
    # (otherwise, index numbers will be out of order)
    train.reset_index(inplace=True, drop=True)
    test.reset_index(inplace=True, drop=True)

    # Write train and test DataFrames to output csv files with same name as input csv file with
    # _train.csv or _test.csv appended
    train_file_path = os.path.splitext(subset_meta_dose_hr_csv_path)[0] + "_train.csv"
    test_file_path = os.path.splitext(subset_meta_dose_hr_csv_path)[0] + "_test.csv"

    train.to_csv(train_file_path)
    test.to_csv(test_file_path)

    return (train_file_path, test_file_path)

In [ ]:
train_test_split_subset_meta_dose_hr(
    subset_meta_dose_hr_csv_path=subset_new_path_fname,
    test_size=0.2,
    out_dir_csv=output_dir,
    random_state=42,
    stratify_col="particle_type")

#TODO
- save tiffs locally from the aws bucket

# Clone repository

In [ ]:
!git clone https://github.com/UC-Irvine-CS175/final-project-saddleback

# Change directory into your repository and checkout to Your Working Branch if it's not main

In [ ]:
%cd final-project-saddleback

In [ ]:
# !git checkout the-working-branch

In [ ]:
# Confirm that your working branch is up to date
!git log --oneline -5

In [ ]:
# Pull if your working branch is not up to date
!git pull

In [ ]:
# For this command to work you must have requirements.txt present in the current working directory
!pip install -r requirements.txt

In [ ]:
# For system agnostic paths, we use pyprojroot to set a root
import os              # os.path.join()
import pyprojroot      # pyprojroot.here()

# Appends the repository's base directory to the list of module paths
root = pyprojroot.find_root(pyprojroot.has_dir(".git"))
import sys
sys.path.append(str(root))

# Important - Our BPSConfig class
3. `data_dir`: The directory we have our raw data saved to. Since we downloaded and un-tar'd the dataset before entering our repository, we need to properly access it from `../processed`
4. `train_meta_fname`: The name of the csv file holding all the filenames and labels for the training dataset
    - You will need to change this if you created your own subset
5. `val_meta_fname`: The name of the csv file holding all the filenames and labels for the validation dataset
    - You will need to change this if you created your own subset
6. `save_dir`: The directory where we want to save our model weights
7. `batch_size`: Number of images per batch
8. `max_epochs`: Number of epochs for training/validation
9. `accelerator`: Type of device we would like to use (CPU, GPU, TPU, etc)
10. `devices`: The number of devices for training. Leave at one unless you know what you're doing
11. `num_workers`: The number of workers from your CPU that will be preparing the dataloader for the model. For free Colab tier, try 8 at first.
12. `seed`: An integer that determines the outputs of randomness. Makes sure that your training results are reproducible for others to check.
13. `img_size`: Size of an image for our custom resize function. The pretrained models used in this example will automatically resize the images if they are not resized, but we have included this for scalability.
14. `model`: A string that is the name of the model. It must exactly match the names in `torch.hub` [here](https://pytorch.org/vision/stable/models.html#classification).

In [ ]:
@dataclass
class BPSConfig:
    data_dir:           str = '../processed'
    train_meta_fname:   str = 'meta_dose_hi_hr_4_post_exposure_train.csv'
    val_meta_fname:     str = 'meta_dose_hi_hr_4_post_exposure_test.csv'
    save_dir:           str = os.path.join(root, 'models', 'pretrained')
    batch_size:         int = 16
    max_epochs:         int = 5
    accelerator:        str = 'auto'
    devices:            int = 1
    num_workers:        int = 8
    seed:               int = 42
    img_size:           int = 256
    model:              str = 'vgg11_bn'

bps_config = BPSConfig()

## Requirements of our pre-trained Models
The pre-trained model architectures we are downloading from torch.hub require some additional preprocessing for the model to accept them as inputs.

1. Normalization
  - The tif file images are uint16, meaning that the dynamic range of the pixels are between 0 and 2<sup>16</sup>, but to be useful for our model and our interpretation, we need to normalize those values as a float between 0 and 1
2. Resize
  - The pretrained models we get from torch.hub will resize and crop the images for us, but it is good practice to resize the images ourselves to maintain further transfer learning with other model architectures in the future
3. Three channel RGB images.
  - We set the single channel image to a three channel image by copying the greyscale image to three channels.
4. Conversion to Tensors
  - The model expects a Tensor as input, so this function will convert the numpy arrays to Tensors.
5. Dataloader
  - The dataloader will compose these preprocessing transformations in sequence and feed them into the model in the proper batch size indicated in our BPSConfig class.

In [ ]:
from src.dataset.augmentation import(
    NormalizeBPS,
    ResizeBPS,
    ToThreeChannels,
    ToTensor
)
from src.dataset.bps_datamodule import BPSDataModule

In [ ]:
import pytorch_lightning as pl

# Fix random seed
pl.seed_everything(bps_config.seed, workers=True)

# Instantiate our BPSDataModule ⌛️

In [ ]:
bps_datamodule = BPSDataModule(train_csv_file=bps_config.train_meta_fname,
                               train_dir=bps_config.data_dir,
                               val_csv_file=bps_config.val_meta_fname,
                               val_dir=bps_config.data_dir,
                               resize_dims=(bps_config.img_size, bps_config.img_size),
                               batch_size=bps_config.batch_size,
                               num_workers=bps_config.num_workers)

# Using BPSDataModule's setup, define the stage name ('train' or 'validate')
bps_datamodule.setup(stage='train')       #training set dataloader
bps_datamodule.setup(stage='validate')    #validation set dataloader

# Import Modules 🐍 & Load Pre-trained Model Weights 🏋

The default weights are selected here, which will always grab the best performing weights from torch.hub.

In [ ]:
import torch
from torch.optim import Adam
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transform, utils
from torchmetrics import Accuracy

import numpy as no
import random

import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from pytorch_lightning import LightningModule
from datetime import datetime

In [ ]:
my_model = torch.hub.load('pytorch/vision',
                          bps_config.model,
                          weights='DEFAULT')

# Modify the last layers for particle type classification

The pretrained models have different architecture that match the outputs for the data they were trained on. For our example, we need to make sure they output 2 probabilities because our model should be specified for binary classification.

In [ ]:
if bps_config.model in ['resnet18', 'resnet50', 'resnet101']
  num_features = my_model.fc.in_features
  my_model.fc = nn.Linear(num_features, 2)
  isResNet = True
  isVGG = False
  isSqueezeNet = False
elif bps_config.model == 'vgg11_bn'
  num_features = my_model.classifier[6].in_features
  my_model.classifier[6] = nn.Linear(num_features, 2)
  isVGG = True
  isResNet = False
  isSqueezeNet = False
elif bps_config.model == 'squeezenet1_1'
  final_conv = nn.Conv2d(512, 2, kernel_size=1)
  my_model.classifier._modules['1'] = final_conv
  my_model.num_classes = 2
  isSqueezeNet = True
  isResNet = False
  isVGG = False

# Freezing layers
An important part of tuning the model is experimenting with the pre-trained layers. The model will still make use of the pre-trained weights in the frozen layers, but the gradients are not updated during training.

This is not required for training, and you can experiment with unfreezing some/all layers, or going even further an adding new layers. This will not be covered here, but just know that you can comment out the following code cell if you want every layer in the model architecture to train on your dataset, instead of the final layer that outputs a Tensor of probabilities we use to generate a prediction.

In [ ]:
if isResNet
  for name, param in my_model.named_parameters():
      if "fc" not in name:
          param.requires_grad_(False)  # Set requires_grad to False

elif (isVGG or isSqueezeNet)
  for name, param in my_model.named_parameters():
      if "classifier" not in name:
          param.requires_grad_(False)  # Set requires_grad to False

# The BPS Classifier Class Definition ⚡
Pytorch Lightning abstracts some of the important concepts that we take for granted, for example the training loop itself. The general premise is that since there are some things that are always present, we can have that taken care of in the background and just focus on overriding parts of the process that are specific to our implementation. See [this link](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#hooks) to go deeper and read the pseudocode. You can override any of the functions to make the model do exactly what you want, when you want.

## `__init__`
- Define a model, a loss function, an accuracy function, a learning rate, and how you would like to save hyperparameters.

## `forward`
- The "forward pass" takes an input and passes it through the layers in your model. Pytorch Lightning lets us pass in pretrained models from torch.hub and utilize our dataloader pipeline without any customization needed.

## `training_step`
- Loop through all our batches over the course of an epoch, saving the loss and logging it per-epoch. Pytorch Lightning abstracted away the actual loop in another function that we are not overriding, because it will often be the same across a wide variety of Machine Learning solutions.

## `validation_step`
- Similar to the training_step, but in this step we need to see how accurate the model is after learning from the previous training step(s).

## `test_step`
- In the BPSClassifier, we do not have access to the test data, but it is provided for future use in case it is released or if the dataset author wishes to use it here.

## `configure_optimizers`
- Set an optimizer (we use Adam here) with a learning rate. You can add a lr scheduler here as well.


In [ ]:
class BPSClassifier(pl.LightningModule):
    def __init__ (self, model, n_classes=2, lr=1e-3):

        super().__init__()
        self.model = model

        self.loss_fn = CrossEntropyLoss()
        self.accuracy_fn = Accuracy(task='binary', num_classes=2)
        self.lr = lr
        self.save_hyperparameters

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(y)

        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(y)

        y_pred = torch.argmax(y_hat, dim=1)
        acc = self.accuracy_fn(y_pred, y)

        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_accuracy", acc, on_step=False, on_epoch=True)

        return preds

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(y)

        loss = self.loss_fn(y_hat, y)

        y_pred = torch.argmax(y_hat, dim=1)
        acc = self.accuracy_fn(y_pred, y)

        self.log("test_loss", loss)
        self.log("test_accuracy", acc)

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.lr)
        return optimizer

# Create a wandb session 🪄
- The next cell will create a link for you the copy your authorization code. Paste that into the terminal output in the cell below after you run it so that you can upload your results to Weights & Biases.

In [ ]:
!wandb login --relogin

In [ ]:
# Create a unique name for your run that is descriptive
# We set the mode to 'online' so that the weights are not saved locally (we will use Pytorch Lightning for that)
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
wandb_logger = WandbLogger(project='deeplearning-eda-saddleback',
                           log_model="all",
                           entity='saddleback',
                           name=f'{bps_config.model}_{bps_config.max_epochs}_{current_datetime}',
                           mode='online')

# Time to train!

In [ ]:
# Pass pre-trained model to the BPSClassifier to define the forward pass
my_model = BPSClassifier(my_model)

# Initialize the trainer, and include a directory where to save model weights with Pytorch Lightning
trainer = Trainer(logger=wandb_logger,
                  accelerator=bps_config.accelerator,
                  max_epochs=bps_config.max_epochs,
                  default_root_dir='path/to/save/checkpoints')

# Training Time!
trainer.fit(model=my_model,
            train_dataloader=bps_datamodule.train_dataloader(),
            val_dataloder=bps_datamodule.val_dataloder())

In [ ]:
wandb.finish()

# Loading Weights from Training

In [ ]:
model = MyLightningModule.load_from_checkpoint('path/to/save/checkpoints/example.ckpt')

# The world is your oyster!
You can use this model however you want now.

<img src="https://media.giphy.com/media/MCZ39lz83o5lC/giphy.gif" height=200></img>